Importing Relevant Libraries

In [242]:
import sys
import os

In [243]:
import pydicom
import pandas as pd
import SimpleITK as sitk
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms

Helper functions for Loading Series
TODO: EXPAND TO FULL SET

In [244]:
# Function to load DICOM series
def load_dicom_series(series_dir, target_size=(224, 224)):
    """Load and normalize a DICOM series, return (1, H, W)"""
    """    #This is within the single folder-- this is within a single series (ex. 01-01-1990-NA-MRI BREAST BILATERAL WWO-97538\26.000000-ax t1 tse c-58582)
"""
    slices = []
    for file in sorted(os.listdir(series_dir)):
        if file.endswith('.dcm'):
            dcm = pydicom.dcmread(os.path.join(series_dir, file))
            slices.append(dcm.pixel_array.astype(np.float32))
    volume = np.stack(slices, axis=0)

    # Collapse Z
    image = np.mean(volume, axis=0)
    image = (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-5)

    image = torch.tensor(image).unsqueeze(0)  # (1, H, W)
    image = torch.nn.functional.interpolate(image.unsqueeze(0), size=target_size, mode='bilinear', align_corners=False)
    return image.squeeze(0)  # (1, H, W)

In [245]:
# Function to load DICOM series seg
def load_nrrd_mask(nrrd_path):
    #all nrrd should be available, something has gone wrong if not
    """Load NRRD segmentation mask and return binary mask tensor (1, H, W)"""
    if not os.path.exists(nrrd_path):
        return None
    image = sitk.ReadImage(nrrd_path)
    array = sitk.GetArrayFromImage(image)  # shape: (Z, H, W)
    mask = (array > 0).astype(np.float32)
    if mask.shape[0] > 1:
        mask = np.mean(mask, axis=0)
    else:
        mask = mask[0]
    return torch.tensor(mask).unsqueeze(0)  # shape: (1, H, W)

In [246]:
#Sanity check for dicom paths being real
test_dicom_path = r"D:\brc\image\manifest-1654812109500\Duke-Breast-Cancer-MRI\Breast_MRI_001\01-01-1990-NA-MRI BREAST BILATERAL WWO-97538\26.000000-ax t1 tse c-58582"
print(os.path.exists(test_dicom_path))
testSeriesLoadStack = load_dicom_series(test_dicom_path)
print(testSeriesLoadStack)

#Sanity check for seg data
test_dicom_seg_path = r"D:\brc\seg\3dtest\PKG - Duke-Breast-Cancer-MRI-Supplement-v3\Duke-Breast-Cancer-MRI-Supplement-v3\Segmentation_Masks_NRRD\Breast_MRI_002\Segmentation_Breast_MRI_002_Breast.seg.nrrd"
print(os.path.exists(test_dicom_seg_path))
testDicomLoadSeries = load_nrrd_mask(test_dicom_seg_path)
print(testDicomLoadSeries)

True
tensor([[[2.5396e-05, 3.7003e-04, 4.0368e-04,  ..., 3.1659e-04,
          3.5772e-04, 3.5819e-04],
         [3.6816e-04, 5.3052e-03, 5.6469e-03,  ..., 4.4556e-03,
          5.0408e-03, 4.9562e-03],
         [4.0119e-04, 5.5984e-03, 5.5269e-03,  ..., 4.4642e-03,
          5.2574e-03, 4.7046e-03],
         ...,
         [7.9194e-03, 1.1006e-01, 1.0387e-01,  ..., 1.3180e-01,
          1.3895e-01, 1.4201e-01],
         [7.6901e-03, 1.0773e-01, 1.0672e-01,  ..., 1.3339e-01,
          1.3905e-01, 1.4002e-01],
         [7.4593e-03, 1.0429e-01, 1.0424e-01,  ..., 1.3543e-01,
          1.3972e-01, 1.4050e-01]]])
True
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


Test Dataset Class

In [247]:
class BreastMRIDataset(Dataset):
    def __init__(self, series_dirs, mask_paths, labels, transform=None, use_mask=True):
        """
        series_dirs: list of directories with DICOM series
        mask_paths: list of NRRD mask file paths (can be None)
        labels: list of outcome labels
        """
        self.series_dirs = series_dirs
        self.mask_paths = mask_paths
        self.labels = labels
        self.transform = transform
        self.use_mask = use_mask

    def __len__(self):
        return len(self.series_dirs)

    def __getitem__(self, idx):
        image = load_dicom_series(self.series_dirs[idx])  # (1, H, W)
        mask = None
        if self.use_mask and self.mask_paths[idx] is not None:
            mask = load_nrrd_mask(self.mask_paths[idx])  # (1, H, W)

        if self.transform:
            image = self.transform(image)
            if mask is not None:
                mask = self.transform(mask)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, mask, label


Functions for Accessing Training and Test

In [248]:
#pipeline:
#trawl csv file of training ids
#for every one, construct path to where the series is
#for every one, construct path to where the mask (DENSE AND VESSELS) is
#for every one return label

In [255]:
baselineLocation = "D:\\brc\\ids"
baselineLocationImgs = "D:\\brc\\image\\manifest-1654812109500\\Duke-Breast-Cancer-MRI"
baselineLocationSeg = "D:\\brc\\seg\\3dtest\\PKG - Duke-Breast-Cancer-MRI-Supplement-v3\\Duke-Breast-Cancer-MRI-Supplement-v3\\Segmentation_Masks_NRRD"
locationOfClin = "D:\\brc\\clin\\clinical.csv"
print(os.path.exists(locationOfClin))

True


In [250]:
def trawlIdFile():
    dir_list = os.listdir(baselineLocationSeg)
    print(dir_list)
    df = pd.DataFrame(dir_list, columns=['Name'])
    return df

In [251]:
def buildPathToSeries(patient):
    currentDir = os.path.join(baselineLocationImgs, patient)
    folders = [f for f in os.listdir(currentDir) if os.path.isdir(os.path.join(currentDir, f))]
        # Check if there is at least one folder
    if folders:
        # Grab the first folder (im just expecting the one but just in case)
        firstFolder = folders[0]
        currentDir = os.path.join(currentDir, firstFolder)
        currentDir = os.path.join(currentDir, "T1_IMGS")
    return currentDir


In [252]:
def buildPathToNrrd(patient):
    currentDir = os.path.join(baselineLocationSeg, patient)
    #Segmentation_Breast_MRI_018_Breast.seg.nrrd
    string = "Segmentation_" + patient + "_Breast.seg.nrrd"
    currentDir = os.path.join(currentDir, string)
    return currentDir

In [264]:
def trawlMyRecurrences():
    df = pd.read_csv(locationOfClin)
    df['Recurrence'] = pd.to_numeric(df['Recurrence'], downcast='integer', errors='coerce')
    print(df)
    return df

In [263]:
trawlMyRecurrences()

               Name  Recurrence
0    Breast_MRI_001         0.0
1    Breast_MRI_002         0.0
2    Breast_MRI_003         0.0
3    Breast_MRI_004         0.0
4    Breast_MRI_005         0.0
..              ...         ...
917  Breast_MRI_918         0.0
918  Breast_MRI_919         0.0
919  Breast_MRI_920         0.0
920  Breast_MRI_921         0.0
921  Breast_MRI_922         0.0

[922 rows x 2 columns]


In [269]:
def constructSeriesDirAndMaskPaths():
    series_dirs=[]
    mask_paths=[]
    df = trawlIdFile()
    recurrencedf = trawlMyRecurrences()
    filtered_rec = recurrencedf[recurrencedf['Name'].isin(df['Name'])]
    labels = filtered_rec['Recurrence'].values.tolist()
    print(labels)
    for index, row in df.iterrows():
        patient = row['Name']
        #construct path to series
        mgyPath = buildPathToSeries(patient)
        if (os.path.exists(mgyPath)):
            series_dirs.append(mgyPath)
            mask_paths.append(buildPathToNrrd(patient))
    return series_dirs, mask_paths, labels

In [270]:
constructSeriesDirAndMaskPaths()

['Breast_MRI_002', 'Breast_MRI_006', 'Breast_MRI_018', 'Breast_MRI_021', 'Breast_MRI_023', 'Breast_MRI_025', 'Breast_MRI_031', 'Breast_MRI_041', 'Breast_MRI_054', 'Breast_MRI_076', 'Breast_MRI_080', 'Breast_MRI_087', 'Breast_MRI_089', 'Breast_MRI_105', 'Breast_MRI_112', 'Breast_MRI_124', 'Breast_MRI_141', 'Breast_MRI_143', 'Breast_MRI_148', 'Breast_MRI_166', 'Breast_MRI_170', 'Breast_MRI_186', 'Breast_MRI_204', 'Breast_MRI_229', 'Breast_MRI_230', 'Breast_MRI_238', 'Breast_MRI_246', 'Breast_MRI_272', 'Breast_MRI_279', 'Breast_MRI_280', 'Breast_MRI_286', 'Breast_MRI_287', 'Breast_MRI_290', 'Breast_MRI_302', 'Breast_MRI_320', 'Breast_MRI_329', 'Breast_MRI_334', 'Breast_MRI_337', 'Breast_MRI_338', 'Breast_MRI_339', 'Breast_MRI_354', 'Breast_MRI_359', 'Breast_MRI_363', 'Breast_MRI_369', 'Breast_MRI_370', 'Breast_MRI_383', 'Breast_MRI_392', 'Breast_MRI_426', 'Breast_MRI_435', 'Breast_MRI_440', 'Breast_MRI_464', 'Breast_MRI_466', 'Breast_MRI_489', 'Breast_MRI_492', 'Breast_MRI_495', 'Breast_M

In [ ]:
from torchvision.transforms import Resize, Compose
from torch.utils.data import DataLoader

transform = Compose([
    Resize((224, 224)),  # need to see if architecture allows for this
])
series, masks, recLabels = constructSeriesDirAndMaskPaths()
dataset = BreastMRIDataset(
    series_dirs=series,
    mask_paths=masks,
    labels=recLabels,
    transform=transform,
    use_mask=True,
)

loader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
#put in pickl file  post-architecture so this isnt a pain for anyone